In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# load the Tiny Shakespeare dataset
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)

c:\Users\adhit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Size...: 1 MiB [00:01,  1.25s/ MiB][00:01<00:00,  1.24s/ url]
Dl Completed...: 100%|██████████| 1/1 [00:01<00:00,  1.25s/ url]


Dataset tiny_shakespeare downloaded and prepared to C:\Users\adhit\tensorflow_datasets\tiny_shakespeare\1.0.0. Subsequent calls will reuse this data.


In [6]:
# get the text from the dataset
text = next(iter(dataset['train']))['text'].numpy().decode('utf-8')

# create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# numerically represent the characters
text_as_int = np.array([char2idx[c] for c in text])

# create training examples and targets
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
# batch size and buffer size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [9]:
# length of the vocabulary
vocab_size = len(vocab)

# the embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [10]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [11]:
import os

# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# train the model
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10



155/155 [==============================] - 199s 1s/step - loss: 2.6292
Epoch 2/10
155/155 [==============================] - 192s 1s/step - loss: 1.9031
Epoch 3/10
155/155 [==============================] - 191s 1s/step - loss: 1.6510
Epoch 4/10
155/155 [==============================] - 189s 1s/step - loss: 1.5166
Epoch 5/10
155/155 [==============================] - 189s 1s/step - loss: 1.4345
Epoch 6/10
155/155 [==============================] - 188s 1s/step - loss: 1.3769
Epoch 7/10
155/155 [==============================] - 188s 1s/step - loss: 1.3309
Epoch 8/10
155/155 [==============================] - 191s 1s/step - loss: 1.2906
Epoch 9/10
155/155 [==============================] - 189s 1s/step - loss: 1.2545
Epoch 10/10
155/155 [==============================] - 190s 1s/step - loss: 1.2184


In [12]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [13]:
def generate_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"QUEEN: So, lets end this"))

QUEEN: So, lets end this pleason? And, sir, turn for you,
when panishman comfidention doman in my so fancy clouds.
Sade him for the -apttroushes, which I should fight--
Lords, not by him, he would not speak it. Strike
And lay load to insinrage to this friar.

LIDY BOLY:
May my Romeo's crisw,
Be shortly she is not through forbid:
To-morrow well.
Why, madam, fit worth thy bearf from hand;
But filly fits mean, though to my God,
Is for a birth, and will therefore plead, he hath the death
Of whose blood tunenes on that beautice; let your
grace, redgare, and be in his looks in each
Friar's spending and his coward, is necessity.

QUEEN MARGARET:
And when the vesses old friend, more office are her-ey,
And onerce--I can sweet sir; for so straight
I am all this woman's title, and so sworn.

EDWABEL:
Thy shore,
Since it is best greater, but the tempest shall
Indle son, yet wilt nothing go to--threw that they
Recelove that of, nurse, wife, come death, abort some thy execution ewers,
Well ears be t